In [1]:
import $ivy.`org.apache.spark::spark-sql:3.5.5`
import $ivy.`org.apache.spark::spark-mllib:3.5.5`
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.WARN)

import $ivy.$
import $ivy.$
import org.apache.log4j.{Level, Logger}

# Spark ML Production

In [2]:
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

val spark = SparkSession
                .builder()
                .master("local[*]")
                .appName("Spark ML Production")
                .config("spark.sql.debug.maxToStringFields", 1024)
                .config("spark.log.level", "WARN")
                .getOrCreate()

import spark.implicits._

spark.sparkContext.setLogLevel("WARN")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/03/13 14:45:04 INFO SparkContext: Running Spark version 3.5.5
25/03/13 14:45:04 INFO SparkContext: OS info Mac OS X, 15.3.2, aarch64
25/03/13 14:45:04 INFO SparkContext: Java version 11.0.26
25/03/13 14:45:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting Spark log level to "WARN".
25/03/13 14:45:05 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


import org.apache.spark.sql._
import org.apache.spark.sql.functions._
spark: SparkSession = org.apache.spark.sql.SparkSession@33d1ef23
import spark.implicits._

## Загружаем данные

In [3]:
val data = spark
        .read
        .option("header", "true")
        .option("inferSchema", "true")
        .csv(s"../../data/BankChurners.csv")

data: DataFrame = [CLIENTNUM: int, Attrition_Flag: string ... 21 more fields]

## Загружаем модель

In [4]:
import org.apache.spark.ml.PipelineModel

val model = PipelineModel.load(s"pipelineModel")

import org.apache.spark.ml.PipelineModel
model: PipelineModel = pipeline_1689ca306aa8

## Вычисляем

In [5]:
val predicted = model.transform(data)

predicted: DataFrame = [CLIENTNUM: int, Attrition_Flag: string ... 37 more fields]

In [6]:
predicted.show(10)

+---------+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+----------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------+--------------+-----------------------+----------------------+-----------------------+---------------------+--------------------+-----------------------------+----------------------------+-----------------------------+---------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|CLIENTNUM|   Attriti

## Проверяем результат

In [7]:
val tp = predicted.filter(($"Attrition_Flag" === "Attrited Customer") and ($"prediction" === 1)).count()
val tn = predicted.filter(($"Attrition_Flag" === "Existing Customer") and ($"prediction" === 0)).count()
val fp = predicted.filter(($"Attrition_Flag" === "Existing Customer") and ($"prediction" === 1)).count()
val fn = predicted.filter(($"Attrition_Flag" === "Attrited Customer") and ($"prediction" === 0)).count()

println(s"Confusion Matrix:\n$tp\t$fp\n$fn\t$tn\n")

Confusion Matrix:
1271	2072
356	6428



tp: Long = 1271L
tn: Long = 6428L
fp: Long = 2072L
fn: Long = 356L

In [8]:
val accuracy = (tp + tn) / (tp + tn + fp + fn).toDouble
val precision = tp / (tp + fp).toDouble
val recall = tp / (tp + fn).toDouble

println(s"Accuracy = $accuracy")
println(s"Precision = $precision")
println(s"Recall = $recall\n")

Accuracy = 0.7602448898982916
Precision = 0.38019742746036495
Recall = 0.7811923786109404



accuracy: Double = 0.7602448898982916
precision: Double = 0.38019742746036495
recall: Double = 0.7811923786109404